# California Housing Price Analysis
## CS 133 Team Project - Data Engineering Artifacts

**Team Members:** Sidharth Krishnaswamy, Rayhann Kwon, Arthur Wong, Andrian Than  
**Professor:** Jelena Gligorijevic

This notebook demonstrates all required data engineering artifacts:
- **Artifact A:** Data Card documentation
- **Artifact B:** Dataset transformation with new columns and missing data handling
- **Artifact C:** Grouping, aggregation, and sorting operations
- **Artifact D:** Visualizations (histograms and bar charts)
- **Artifact E:** Reproducible analysis in a single notebook

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
import warnings
warnings.filterwarnings('ignore')

#Plotting style
plt.style.use('default')
sns.set_palette("husl")


## Data Loading


In [ ]:
# Load Dataset
housing = fetch_california_housing(as_frame=True)
df = housing.frame

# Rename columns for clarity
feature_names = {
    'MedInc': 'median_income',
    'HouseAge': 'housing_median_age', 
    'AveRooms': 'avg_rooms',
    'AveBedrms': 'avg_bedrooms',
    'Population': 'population',
    'AveOccup': 'avg_occupancy',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'MedHouseVal': 'median_house_value'
}

df = df.rename(columns=feature_names)
df.head()


Loading California Housing dataset...


,median_income,housing_median_age,avg_rooms,avg_bedrooms,population,avg_occupancy,latitude,longitude,median_house_value
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


# Artifact A: Data Card

Comprehensive documentation of dataset properties, source, and characteristics.

In [ ]:
print("=" * 60)
print("ARTIFACT A: DATA CARD")
print("=" * 60)

print(f"""
Dataset: California Housing Prices (1990 Census)
Source: Scikit-learn datasets (originally from US Census Bureau)
Shape: {df.shape[0]:,} rows x {df.shape[1]} columns
Time Range: 1990 Census data
Coverage: California, USA

COLUMN DIRECTORY:
""")

# Column descriptions with units and ranges
column_info = {
    'median_income': 'Median household income (in tens of thousands of dollars)',
    'housing_median_age': 'Median age of houses in block group (years)', 
    'avg_rooms': 'Average number of rooms per household',
    'avg_bedrooms': 'Average number of bedrooms per household',
    'population': 'Total population in block group (persons)',
    'avg_occupancy': 'Average household size (persons per household)',
    'latitude': 'Latitude coordinate (degrees)',
    'longitude': 'Longitude coordinate (degrees)',
    'median_house_value': 'Median house value (hundreds of thousands of dollars)'
}

for col, desc in column_info.items():
    if col in df.columns:
        print(f"- {col}: {desc}")
        print(f"  {'Range'}: {df[col].min():.3f} to {df[col].max():.3f}")
        print()

# Key targets and features
print("KEY COLUMNS:")
print("- Target Variable: median_house_value")
print("- Geographic Features: latitude, longitude")
print("- Economic Features: median_income")
print("- Housing Features: housing_median_age, avg_rooms, avg_bedrooms, avg_occupancy")
print("- Demographic Features: population")

ARTIFACT A: DATA CARD

Dataset: California Housing Prices (1990 Census)
Source: Scikit-learn datasets (originally from US Census Bureau)
Shape: 20,640 rows x 9 columns
Time Range: 1990 Census data
Coverage: California, USA

COLUMN DIRECTORY:

- median_income: Median household income (in tens of thousands of dollars)
  Range: 0.500 to 15.000

- housing_median_age: Median age of houses in block group (years)
  Range: 1.000 to 52.000

- avg_rooms: Average number of rooms per household
  Range: 0.846 to 141.909

- avg_bedrooms: Average number of bedrooms per household
  Range: 0.333 to 34.067

- population: Total population in block group (persons)
  Range: 3.000 to 35682.000

- avg_occupancy: Average household size (persons per household)
  Range: 0.692 to 1243.333

- latitude: Latitude coordinate (degrees)
  Range: 32.540 to 41.950

- longitude: Longitude coordinate (degrees)
  Range: -124.350 to -114.310

- median_house_value: Median house value (hundreds of thousands of dollars)
  Rang

In [ ]:
#Missing Data Snapshot
print(f"\nMISSING DATA SNAPSHOT:")
missing_counts = df.isnull().sum()
missing_pct = (missing_counts / len(df)) * 100

if missing_counts.sum() == 0:
    print("No missing values found in dataset")
else:
    print("Missing values by column:")
    for col in df.columns:
        if missing_counts[col] > 0:
            print(f"• {col}: {missing_counts[col]:,} missing ({missing_pct[col]:.2f}%)")

#Data Checks Summary
print(f"\nDATA CHECK SUMMARY:")
print(f"- Duplicate rows: {df.duplicated().sum():,}")
print(f"- Rows with any missing values: {df.isnull().any(axis=1).sum():,}")
print(f"- Data types: {df.dtypes.value_counts().to_dict()}")


MISSING DATA SNAPSHOT:
No missing values found in dataset

DATA CHECK SUMMARY:
- Duplicate rows: 0
- Rows with any missing values: 0
- Data types: {dtype('float64'): 9}


In [ ]:
#Data Quirks
print(f"\nDATA QUIRKS:")
print("- House values capped at $500,000")
print("- Geographic clustering - all data from California only")
print("- 1990 data - may not reflect current housing market patterns")
print("- Block group level data - aggregated, not individual house records")
print("- Average rooms/bedrooms may have outliers from institutional properties")


DATA QUIRKS:
- House values capped at $500,000
- Geographic clustering - all data from California only
- 1990 data - may not reflect current housing market patterns
- Block group level data - aggregated, not individual house records
- Average rooms/bedrooms may have outliers from institutional properties


# Artifact B: Dataset Transformation


In [ ]:
print("=" * 60)
print("ARTIFACT B: DATASET TRANSFORMATION")
print("=" * 60)

df_transformed = df.copy()

print("1. CREATING BOOLEAN MASK COLUMNS")
print("-" * 40)

df_transformed['is_coastal'] = (df['longitude'] > -121.5) & (df['latitude'] < 37.5)
coastal_count = df_transformed['is_coastal'].sum()
print(f"is_coastal: Properties likely near coast ({coastal_count:,} properties, {coastal_count/len(df)*100:.2f}%)")

high_income_threshold = df['median_income'].quantile(0.75)
df_transformed['is_high_income_area'] = (df['median_income'] > high_income_threshold)
print(f"is_high_income_area: Areas with income > ${high_income_threshold:.1f}0k ({df_transformed['is_high_income_area'].sum():,} areas, {df_transformed['is_high_income_area'].sum()/len(df)*100:.2f}%)")

df_transformed['aged_house'] = (df['housing_median_age'] > 10)
print(f"aged_house: Areas with houses > 10 years old ({df_transformed['aged_house'].sum():,} areas, {df_transformed['aged_house'].sum()/len(df)*100:.2f}%)")

df_transformed['new_house'] = (df['housing_median_age'] <= 10)
print(f"new_house: Areas with houses < 10 years old ({df_transformed['new_house'].sum():,} areas, {df_transformed['new_house'].sum()/len(df)*100:.2f}%)")

print(f"\nBoolean flag summary:")
boolean_cols = ['is_coastal', 'is_high_income_area', 'aged_house', 'new_house']
for col in boolean_cols:
    true_count = df_transformed[col].sum()
    print(f"  {col:20}: {true_count:5,} ({true_count/len(df)*100:4.1f}%)")

ARTIFACT B: DATASET TRANSFORMATION
1. CREATING BOOLEAN MASK COLUMNS
----------------------------------------
is_coastal: Properties likely near coast (12,971 properties, 62.84%)
is_high_income_area: Areas with income > $4.70k (5,160 areas, 25.00%)
aged_house: Areas with houses > 10 years old (19,071 areas, 92.40%)
new_house: Areas with houses < 10 years old (1,569 areas, 7.60%)

Boolean flag summary:
  is_coastal          : 12,971 (62.8%)
  is_high_income_area : 5,160 (25.0%)
  aged_house          : 19,071 (92.4%)
  new_house           : 1,569 ( 7.6%)


In [ ]:
print("\n2. TRANSFORMATIONS USING MAP() AND APPLY()")
print("-" * 45)

def categorize_income(income):
    """
    Categorize income levels based on 1990 economic standards.
    Ex: 3.0 = $30k
    """
    if income < 3.0:  #less than $30k
        return 'Low'
    elif income < 6.0:  #$30k-$60k
        return 'Medium' 
    elif income < 10.0:  #$60k-$100k
        return 'High'
    else:  #over 100k
        return 'Very High'

df_transformed['income_category'] = df_transformed['median_income'].apply(categorize_income)
print(f"income_category: Applied function to categorize income levels")
print(f"Distribution: {df_transformed['income_category'].value_counts()}")

#Create occupancy density using map()
occupancy_mapping = {
    'Low': lambda x: x < 3.0,
    'Medium': lambda x: 3.0 <= x < 4.0,
    'High': lambda x: 4.0 <= x < 5.0,
    'Very High': lambda x: x >= 5.0
}

def map_occupancy_density(occupancy):
    """Map occupancy values to density categories"""
    for category, condition in occupancy_mapping.items():
        if condition(occupancy):
            return category
    return 'Unknown'

df_transformed['occupancy_density'] = df_transformed['avg_occupancy'].map(map_occupancy_density)
print(f"\noccupancy_density: Mapped average occupancy to density levels")
print(f"Distribution: {df_transformed['occupancy_density'].value_counts()}")

#Create region based on latitude and longitude
def get_region(lat, lon):
    if lat > 37.5:
        return "Northern California"
    elif lat < 34.0:
        return "Southern California"
    else:
        return "Central California"

df_transformed['region'] = df_transformed.apply(lambda row: get_region(row['latitude'], row['longitude']), axis=1)

#Create property type based on average amount of rooms
def get_property_type(row):
    if row['avg_rooms'] > 6:
        return "Large Family Home"
    elif row['avg_rooms'] < 4:
        return "Apartment/Condo"
    else:
        return "Standard Home"

df_transformed['property_type'] = df_transformed.apply(lambda row: get_property_type(row), axis=1)

print(f"\nRegion distribution: {df_transformed['region'].value_counts()}")
print(f"\nProperty type distribution: {df_transformed['property_type'].value_counts()}")


2. TRANSFORMATIONS USING MAP() AND APPLY()
---------------------------------------------
income_category: Applied function to categorize income levels
Distribution: income_category
Medium       10902
Low           7365
High          2064
Very High      309
Name: count, dtype: int64

occupancy_density: Mapped average occupancy to density levels
Distribution: occupancy_density
Low          12617
Medium        6273
High          1404
Very High      346
Name: count, dtype: int64

Region distribution: region
Central California     8347
Southern California    6158
Northern California    6135
Name: count, dtype: int64

Property type distribution: property_type
Standard Home        12365
Large Family Home     5407
Apartment/Condo       2868
Name: count, dtype: int64


In [ ]:
print("\n3. CATEGORIZING/BUCKETING COLUMNS INTO LEVELS")
print("-" * 48)

# Categorization 1: Age categories using pd.cut() with custom bins
age_bins = [0, 10, 25, 40, 100]
age_labels = ['New', 'Modern', 'Mature', 'Old']
df_transformed['age_category'] = pd.cut(df['housing_median_age'], 
                                       bins=age_bins, 
                                       labels=age_labels, 
                                       include_lowest=True)
print(f"age_category: Bucketed housing age into {len(age_labels)} levels")
print(f"Bins: {age_bins}")
print(f"\nDistribution: {df_transformed['age_category'].value_counts()}")


3. CATEGORIZING/BUCKETING COLUMNS INTO LEVELS
------------------------------------------------
age_category: Bucketed housing age into 4 levels
Bins: [0, 10, 25, 40, 100]

Distribution: age_category
Mature    8132
Modern    7061
Old       3878
New       1569
Name: count, dtype: int64


In [ ]:
print("\n4. MISSING DATA HANDLING")
print("-" * 30)

# Check for missing values in original data
original_missing = df.isnull().sum().sum()
print(f"Missing values in original dataset: {original_missing}")

# Decision rationale
print(f"\nMISSING DATA STRATEGY RATIONALE:")
print(f"No missing values in original dataset")
print(f"If there were missing values, we would have filled them with 'Unknown' or dropped them according to the data type")



4. MISSING DATA HANDLING
------------------------------
Missing values in original dataset: 0

MISSING DATA STRATEGY RATIONALE:
No missing values in original dataset
If there were missing values, we would have filled them with 'Unknown' or dropped them according to the data type


# Artifact C: Grouping, Aggregation & Sorting

In [ ]:
print("=" * 60)
print("ARTIFACT C: GROUPING, AGGREGATION & SORTING")
print("=" * 60)

print("1. VALUE_COUNTS() ANALYSIS")
print("-" * 30)

print("INCOME CATEGORY DISTRIBUTION:")
income_counts = df_transformed['income_category'].value_counts()
for category, count in income_counts.items():
    pct = (count / len(df_transformed)) * 100
    print(f"  {category}: {count:5,} areas ({pct:4.1f}%)")

print(f"\nINTERPRETATION:")
print(f"- Most areas fall in the medium income bracket ($30k-$60k)")
print(f"- This suggests a balanced economic distribution across California regions in 1990")
print(f"- Only {income_counts['Very High']:,} areas had very high income (>$100k), showing income inequality")
print(f"- A good amount of areas ({income_counts['Low']:,}) had low income (<$30k), indicating a lower standard of living")


ARTIFACT C: GROUPING, AGGREGATION & SORTING
1. VALUE_COUNTS() ANALYSIS
------------------------------
INCOME CATEGORY DISTRIBUTION:
  Medium: 10,902 areas (52.8%)
  Low: 7,365 areas (35.7%)
  High: 2,064 areas (10.0%)
  Very High:   309 areas ( 1.5%)

INTERPRETATION:
- Most areas fall in the medium income bracket ($30k-$60k)
- This suggests a balanced economic distribution across California regions in 1990
- Only 309 areas had very high income (>$100k), showing income inequality
- A good amount of areas (7,365) had low income (<$30k), indicating a lower standard of living


In [ ]:
print("\n2. GROUPBY OPERATION 1: Income Analysis")
print("-" * 50)

income_analysis = df_transformed.groupby('income_category').agg({
    'median_house_value': ['mean', 'median', 'min', 'max'],
    'aged_house': ['sum'],
    'avg_occupancy': ['mean', 'min', 'max'],
    'avg_rooms': ['mean', 'min', 'max'],
    'population': ['mean', 'min', 'max'],

}).round(2)

income_analysis.columns = ['_'.join(col).strip() for col in income_analysis.columns]
income_analysis_sorted = income_analysis.sort_values('median_house_value_mean', ascending=False)

print("Housing metrics by income category (sorted by mean house value):")
display(income_analysis_sorted)

print(f"\nKEY INSIGHTS FROM INCOME ANALYSIS:")
highest_income = income_analysis_sorted.index[0]
lowest_income = income_analysis_sorted.index[-1]

house_value_ratio = (income_analysis_sorted.loc[highest_income, 'median_house_value_mean'] / 
                    income_analysis_sorted.loc[lowest_income, 'median_house_value_mean'])

print(f"- {highest_income} income areas have {house_value_ratio:.1f}x higher mean house values than {lowest_income} income areas")
print(f"- Average rooms tend to increase with income: luxury living correlates with space")
print(f"- Higher income areas have more people per household")
print(f"- Higher income areas have less aged houses. High income areas have more new houses")


2. GROUPBY OPERATION 1: Income Analysis
--------------------------------------------------
Housing metrics by income category (sorted by mean house value):


,median_house_value_mean,median_house_value_median,median_house_value_min,median_house_value_max,aged_house_sum,avg_occupancy_mean,avg_occupancy_min,avg_occupancy_max,avg_rooms_mean,avg_rooms_min,avg_rooms_max,population_mean,population_min,population_max
income_category,,,,,,,,,,,,,,
Very High,4.82,5.00,1.12,5.0,272,6.96,1.37,1243.33,8.17,3.17,24.63,1069.92,13.0,7460.0
High,3.63,3.53,0.68,5.0,1805,3.00,1.40,230.17,6.94,2.89,22.82,1435.21,17.0,16122.0
Medium,2.19,1.99,0.15,5.0,9915,2.98,1.28,599.71,5.59,0.89,132.53,1461.31,8.0,16305.0
Low,1.34,1.15,0.15,5.0,7079,3.05,0.69,63.75,4.66,0.85,141.91,1384.63,3.0,35682.0



KEY INSIGHTS FROM INCOME ANALYSIS:
- Very High income areas have 3.6x higher mean house values than Low income areas
- Average rooms tend to increase with income: luxury living correlates with space
- Higher income areas have more people per household
- Higher income areas have less aged houses. High income areas have more new houses


In [ ]:
print("\n3. GROUPBY OPERATION 2: Geographic-Income Analysis")
print("-" * 65)

#Multi-column grouping:
region_income_analysis = df_transformed.groupby(['region', 'income_category']).agg({
    'median_house_value': ['mean', 'count'],
    'population': 'sum',
    'is_coastal': lambda x: (x == True).sum(),
    'is_high_income_area': lambda x: (x == True).sum(),
    'avg_rooms': 'mean',
    'housing_median_age': 'mean'
}).round(2)

region_income_analysis.columns = ['_'.join(col).strip() for col in region_income_analysis.columns]
region_income_analysis_reset = region_income_analysis.reset_index()

region_income_sorted = region_income_analysis_reset.sort_values(
    'median_house_value_mean', ascending=False
)

print("Top 15 region-income combinations by house value:")
display(region_income_sorted.head(15))

print(f"\nGEOGRAPHIC-ECONOMIC INSIGHTS:")

#Most expensive combinations
top_combo = region_income_sorted.iloc[0]
print(f"- Most expensive: {top_combo['region']} region with {top_combo['income_category']} income")
print(f"- Average house value: ${top_combo['median_house_value_mean']:.1f}00k")
print(f"- Coastal properties: {top_combo['is_coastal_<lambda>']:,}")

#Geographic-premium analysis
regional_premiums = region_income_sorted.groupby('region')['median_house_value_mean'].mean().sort_values(ascending=False)
print(f"\nGEOGRAPHIC-ECONOMIC PREMIUMS:")
for region, avg_value in regional_premiums.items():
    print(f"- {region}: ${avg_value:.1f}00k average")

#Impact of location on housing values
print(f"\nTAKEAWAYS & INSIGHTS:")
print(f"- Geographic location significantly impacts house values within same income brackets")
print(f"- Coastal proximity creates additional premium beyond income effects")
print(f"- Bay Area consistently shows highest values across income categories")


3. GROUPBY OPERATION 2: Geographic-Income Analysis
-----------------------------------------------------------------
Top 15 region-income combinations by house value:


,region,income_category,median_house_value_mean,median_house_value_count,population_sum,is_coastal_<lambda>,is_high_income_area_<lambda>,avg_rooms_mean,housing_median_age_mean
3,Central California,Very High,4.88,167,168174.0,104,167,8.13,33.63
11,Southern California,Very High,4.84,84,103713.0,84,84,8.25,19.37
7,Northern California,Very High,4.64,58,58719.0,0,58,8.19,31.97
0,Central California,High,3.83,870,1256204.0,547,870,6.88,27.41
4,Northern California,High,3.48,464,644642.0,0,464,6.97,26.56
8,Southern California,High,3.48,730,1061418.0,730,730,6.99,21.14
2,Central California,Medium,2.27,4254,6481147.0,3321,1110,5.47,27.94
10,Southern California,Medium,2.17,3452,5155930.0,3452,966,5.36,27.00
6,Northern California,Medium,2.10,3196,4294081.0,0,711,5.98,29.18
1,Central California,Low,1.40,3056,4610022.0,2841,0,4.45,30.90



GEOGRAPHIC-ECONOMIC INSIGHTS:
- Most expensive: Central California region with Very High income
- Average house value: $4.900k
- Coastal properties: 104

GEOGRAPHIC-ECONOMIC PREMIUMS:
- Central California: $3.100k average
- Southern California: $3.000k average
- Northern California: $2.900k average

TAKEAWAYS & INSIGHTS:
- Geographic location significantly impacts house values within same income brackets
- Coastal proximity creates additional premium beyond income effects
- Bay Area consistently shows highest values across income categories


# Artifact D: Advanced Scatter Plot Visualizations

In [ ]:
#histogram
numeric_target = "median_house_value" if "median_house_value" in df.columns else "median_income"

ax = df[numeric_target].plot(kind="hist", bins=40)
ax.set_title(f"Distribution of {numeric_target}")
ax.set_xlabel(numeric_target)
ax.set_ylabel("Count")
ax.grid(False)
plt.show()

In [ ]:
#frequency chart
bins   = [0, 10, 20, 30, 40, 50, 60]
labels = ["0–10", "10–20", "20–30", "30–40", "40–50", "50–60"]

age_bins = pd.cut(df["housing_median_age"], bins=bins, labels=labels, right=False, include_lowest=True)
freq = age_bins.value_counts().sort_index()

ax = freq.plot(kind="bar")
ax.set_title("Frequency of House Age Bins")
ax.set_xlabel("House Age (years)")
ax.set_ylabel("Number of tracts")
ax.tick_params(axis="x", rotation=0)  # keep ticks readable
ax.grid(False)
plt.show()

# Artifact E: Reproduceability